In [13]:
import os
import shutil
import requests
import zipfile
import subprocess
import json
from tqdm import tqdm
import time
from tempfile import TemporaryDirectory

In [ ]:

# -------- CONFIG --------
# https://github.com/ChatGPTNextWeb/NextChat
REPO_OWNER = "ChatGPTNextWeb"
REPO_NAME = "NextChat"
GITHUB_TOKEN = ''
MAX_PAGES = 610  # adjust if needed
RESULTS_FILE = "matches.jsonl"
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"} if GITHUB_TOKEN else {}
# ------------------------

In [5]:
API_URL = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/forks"

In [26]:
61000/100

610.0

In [50]:
import requests
import time
import json
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# --- Config ---
API_URL = "https://api.github.com/repos/ChatGPTNextWeb/NextChat/forks"
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"}
MAX_PAGES = 610
SAVE_FILE = "fork_names.json"

# --- Retry Setup ---
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=2,  # exponential backoff: 1s, 2s, 4s, 8s, 16s
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("https://", HTTPAdapter(max_retries=retries))

fork_names = []

for page in range(1, MAX_PAGES + 1):
    try:
        print(f"[Page {page}] Fetching forks...")
        response = session.get(API_URL, headers=HEADERS, params={"per_page": 100, "page": page}, timeout=15)

        if response.status_code != 200:
            print(f"⚠️  Non-200 status code: {response.status_code} - stopping")
            break

        page_forks = response.json()

        if not page_forks:
            print("✅ No more forks found — ending early.")
            break

        new_forks = [fork["full_name"] for fork in page_forks]
        fork_names.extend(new_forks)
        print(f"✅ Found {len(new_forks)} forks (total: {len(fork_names)})")

        # Optional: save progress
        if page % 10 == 0:
            with open(SAVE_FILE, "w") as f:
                json.dump(fork_names, f)

        # Throttle
        time.sleep(1)
        if page % 4 == 0:
            time.sleep(6)

    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed on page {page}: {e}")
        print("🕒 Backing off for 30 seconds before retrying this page...")
        time.sleep(30)
        continue  # try the same page again

# Final save
with open(SAVE_FILE, "w") as f:
    json.dump(fork_names, f)

print(f"\n🎉 Done! Total forks collected: {len(fork_names)}")


[Page 1] Fetching forks...
✅ Found 100 forks (total: 100)
[Page 2] Fetching forks...
✅ Found 100 forks (total: 200)
[Page 3] Fetching forks...
✅ Found 100 forks (total: 300)
[Page 4] Fetching forks...
✅ Found 100 forks (total: 400)
[Page 5] Fetching forks...
✅ Found 100 forks (total: 500)
[Page 6] Fetching forks...
✅ Found 100 forks (total: 600)
[Page 7] Fetching forks...
✅ Found 100 forks (total: 700)
[Page 8] Fetching forks...
✅ Found 100 forks (total: 800)
[Page 9] Fetching forks...
✅ Found 100 forks (total: 900)
[Page 10] Fetching forks...
✅ Found 100 forks (total: 1000)
[Page 11] Fetching forks...
✅ Found 100 forks (total: 1100)
[Page 12] Fetching forks...
✅ Found 100 forks (total: 1200)
[Page 13] Fetching forks...
✅ Found 100 forks (total: 1300)
[Page 14] Fetching forks...
✅ Found 100 forks (total: 1400)
[Page 15] Fetching forks...
✅ Found 100 forks (total: 1500)
[Page 16] Fetching forks...
✅ Found 100 forks (total: 1600)
[Page 17] Fetching forks...
✅ Found 100 forks (total: 1700

In [9]:
import pandas as pd
df = pd.read_json('fork_names.json')
fork_names = df[0].tolist()

In [21]:
import requests
import time
import zipfile
import os
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Setup a session with retry logic
zip_session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=2,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
zip_session.mount("https://", HTTPAdapter(max_retries=retries))

def download_and_extract_zip(repo_full_name, extract_to):
    owner, repo = repo_full_name.split("/")
    NO_HEADERS = {}  # not needed for public ZIPs
    branches = ["main", "master"]

    for branch in branches:
        zip_url = f"https://github.com/{owner}/{repo}/archive/refs/heads/{branch}.zip"
        try:
            # print(f"⬇️  Downloading ZIP for {repo_full_name} ({branch})...")
            res = zip_session.get(zip_url, headers=NO_HEADERS, stream=True, timeout=30)

            if res.status_code != 200:
                print(f"⚠️  Could not fetch {branch}.zip for {repo_full_name} (status {res.status_code})")
                continue

            zip_path = os.path.join(extract_to, "repo.zip")
            with open(zip_path, "wb") as f:
                for chunk in res.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_to)
                # print(f"✅ Successfully extracted {repo_full_name} ({branch})")
                return branch  # return the branch that worked
            except Exception as e:
                print(f"❌ Failed to extract ZIP for {repo_full_name}: {e}")
                return None

        except requests.exceptions.RequestException as e:
            print(f"❌ Connection error on {repo_full_name} ({branch}): {e}")
            print("🕒 Backing off for 20 seconds before retry...")
            time.sleep(20)
            continue

    # print(f"❌ Failed to download any branch for {repo_full_name}")
    with open('repo_code_not_found.txt','a') as f:
        f.write(f'{repo_full_name}\n')

    return None


def search_with_ripgrep(search_root, search_string):
    try:
        exclude_globs = [
            "!*.{png,jpg,jpeg,svg,ico,webp,bmp,gif,mp4,mp3,m4a,ogg,avi,mov,mkv,flac}",
            "!*.{woff,woff2,ttf,otf,pdf}",
            "!*.{zip,tar.gz,tgz,rar,7z,gz,bz2,xz}",
            "!*.{min.js,min.css,map,class,o,wasm,dll,exe,so}",
            "!*.{lock,csv,tsv,db,sqlite,log,bak,swp,tmp}",
            "!*~",
            "--glob=!node_modules/**",
            "--glob=!.next/**",
            "--glob=!dist/**",
            "--glob=!build/**",
            "--glob=!coverage/**",
            "--glob=!.git/**",
        ]

        rg_command = [
            "rg", "--fixed-strings", "--no-heading", "--color", "never",
            search_string, search_root
        ] + exclude_globs

        result = subprocess.run(rg_command, capture_output=True, text=True)
        raw_output = result.stdout.strip()
        if not raw_output:
            return []

        file_set = set()
        for line in raw_output.splitlines():
            if ":" in line:
                abs_path, _ = line.split(":", 1)
                rel_path = os.path.relpath(abs_path, search_root)
                file_set.add(rel_path)

        return sorted(file_set)
    except Exception as e:
        print(f"ripgrep failed in {search_root}:", e)
        return []


In [22]:
fork_names[0]

'gitgitTrue/NextChat'

In [23]:
print(f"Found {len(fork_names)} forks")

# FULL_SYSTEM_PROMPT = '''**Casual Style**
#                 2. **Detailed Responses with Emphasis** - Ensure thoroughness and depth in explanations, covering all relevant aspects.
#                 3. **Neutral, Suggestive When Clear**
#                 4. **Use LaTeX for Math When Applicable** - Apply only for math-related queries, Inline: `$equation$`, Display: `$$equation$$`
#                 5. **Match Query Language Precisely** - If the question is in Chinese, respond in Chinese; for other languages, the same rule applies.
#                 6. **Focus on Specifics**'''

SYSTEM_PROMPT = 'If the question is in Chinese, respond in Chinese; for other languages, the same rule applies.'
df = pd.read_json('matches.jsonl',lines=True)
already_searched = df['repo'].tolist()
# already_searched = []
with open(RESULTS_FILE, "a") as out_file:
    for batch in range(0,61_000,500):
        print(f'Batch: {batch}')
        for fork in tqdm(fork_names[batch:batch+500], desc="Processing forks"):
            if fork in already_searched:
                # print(f'{fork} already searched')
                continue
            with TemporaryDirectory() as tmpdir:
                MAIN_OR_MASTER = download_and_extract_zip(fork, tmpdir)
                if not MAIN_OR_MASTER:
                    continue
                subdirs = [d for d in os.listdir(tmpdir) if os.path.isdir(os.path.join(tmpdir, d))]
                if not subdirs:
                    continue
                repo_dir = os.path.join(tmpdir, subdirs[0])
                matches = search_with_ripgrep(repo_dir, SYSTEM_PROMPT)
                urls = [
                    f"https://github.com/{fork}/blob/{MAIN_OR_MASTER}/{path}"
                    for path in matches
                ]

                out_file.write(json.dumps({
                    "repo": fork,
                    "files": urls}) + "\n")
                
                if len(matches) > 0:
                    print(f'Matches found in {fork}')
                
    print(f"\n✅ Done! See results in {RESULTS_FILE}")

Found 60995 forks
Batch: 0


Processing forks:  90%|█████████ | 450/500 [00:00<00:00, 1062.17it/s]

⚠️  Could not fetch main.zip for 20250315lh/NextChat (status 404)
⚠️  Could not fetch master.zip for 20250315lh/NextChat (status 404)


Processing forks: 100%|██████████| 500/500 [00:00<00:00, 649.19it/s] 


⚠️  Could not fetch main.zip for tim-coding-zero/NextChat (status 404)
⚠️  Could not fetch master.zip for tim-coding-zero/NextChat (status 404)
Batch: 500


...

Batch: 60500


Processing forks:  63%|██████▎   | 310/495 [00:00<00:00, 3092.14it/s]

⚠️  Could not fetch main.zip for xhazy01/ChatGPT-Next-Web (status 404)
⚠️  Could not fetch master.zip for xhazy01/ChatGPT-Next-Web (status 404)
⚠️  Could not fetch main.zip for esgcopilot/ChatGPT-Next-Web (status 404)
⚠️  Could not fetch master.zip for esgcopilot/ChatGPT-Next-Web (status 404)
⚠️  Could not fetch main.zip for szukodf/ChatGPT-Next-Web-1 (status 404)
⚠️  Could not fetch master.zip for szukodf/ChatGPT-Next-Web-1 (status 404)


Processing forks: 100%|██████████| 495/495 [00:01<00:00, 311.98it/s] 

⚠️  Could not fetch main.zip for JoaoAraujo40/test (status 404)
⚠️  Could not fetch master.zip for JoaoAraujo40/test (status 404)

✅ Done! See results in matches.jsonl
